Using snorkel to classify articles about oil markets versus all others.

One apparent finding from this and the Syria example (`snorkel_testing_arabic_3_syria.ipynb`) is that a short list of terms that you're very confident will appear in the text is very powerful. Here, that's oil and related terms.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import snorkel

%matplotlib inline
from IPython.core.pylabtools import figsize

Important!! Snorkel assumes 1 for positive labels, 0 for negative labels, -1 for abstaining. I was getting terrible results by flipping the 0 and -1.

In [2]:
from snorkel.labeling.apply import PandasLFApplier
from snorkel.labeling.lf import labeling_function

POS = 1
NEG = 0
ABSTAIN = -1

In [3]:
#Load in our cleaned data and re-check category labels
media_df = pd.read_csv('/Users/awhite/Documents/snorkel/arabic_news_cleaned.csv')
media_df.category.value_counts()

كرة_القدم                       4301
الأزمة_السورية                 3160
جماعات_مسلحة                    2142
أسواق_النفط                    1364
لاجئون                         1256
رياضات_اخرى                     1100
المعارضة_السورية                1002
صواريخ                           741
التقنية_والمعلومات               639
فضاء                             630
أسلحة_ومعدات_عسكرية             625
الهجرة_إلى_أوروبا              616
الأزمة_الأوكرانية              595
اكتشافات                         577
الأزمة_اليمنية                  566
تفجيرات                          559
مشاهير                           539
البحوث_الطبية                    530
معلومات_عامة                     484
طائرات_حربية                    479
الانتخابات_الأمريكية            447
جرائم                           433
مظاهرات                          422
مؤشرات_اقتصادية                 400
عقوبات_اقتصادية                  397
الاعتراف_بدولة_فلسطين            369
امراض                            323
أولمبياد_ريو_د

There are several broad categories like جماعات_مسلحة (armed groups) and أسواق_النفط (oil markets), then more specific categories such as two for the Syrian conflict: الأزمة_السورية (Syrian conflict) and المعارضة_السورية (Syrian opposition). Let's start with one of the general categories, then move on to Syria.  

In [4]:
#Recoding data for these two classification problems
#Full category names not working, probably because Arabic

media_df = media_df.assign(oil = media_df.category.str.contains("النفط") == True)
media_df = media_df.assign(syria = media_df.category.str.contains("سورية") == True)

media_df.oil = media_df.oil.replace({True:1,False:0})
media_df.syria = media_df.syria.replace({True:1,False:0})

media_df[media_df.oil == 1].head()

,text,category,oil,syria
6208,اسعار تتراجع توقعات انتاج اوبك مستوياته حاليه ...,أسواق_النفط,1,0
6209,تبدا ببنا انابيب سيبيريا روسي اراض قالت شركه غ...,أسواق_النفط,1,0
6210,اسعار ترتفع بدعم ارتفعت اسعار بدعم نزوله تعامل...,أسواق_النفط,1,0
6211,اوبك تجتمع فيينا توقعات ابقا مستويات انتاج تجت...,أسواق_النفط,1,0
6212,روسيا تنوي محافظه مستويات انتاج حاليه طاقه روس...,أسواق_النفط,1,0


In [5]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(media_df, test_size = 0.2, random_state = 0)

train, valid = train_test_split(train, test_size = 0.2, random_state = 0)
train, dev = train_test_split(train, test_size = 0.2, random_state = 0)

Y_train = train["oil"].values
Y_dev = dev["oil"].values
Y_valid = valid["oil"].values
Y_test = test["oil"].values

len(train)

14166

In [6]:
pd.set_option("display.max_colwidth", 0)

dev[dev.oil == 1].sample(5)

,text,category,oil,syria
6441,ايران تستعد لاستئناف شحنات اوروبا تخطط طهران لشحن برميل يوميا اوروبيه اصبحت شركه جلينكور شركه غربيه تتعامل ايراني اظهرت بيانات لتتبع ناقله استاجر جلينكور شركه انجلو سويسريه متخصصه تجاره اوليه تحمي بنحو ايراني ماضي موقع معلومات زاره ايرانيه انترنت شانا ايراني بيجن زنغنه قوله مبيعات ايراني لاوروبا عقوبات دوليه طهران تجاوزت برميل اضاف وزير ايراني طهران تخطط لزياده صادرات بمقدار برميل يوميا اشهر قادمه صادرات ايرانيه ستتجه اسواق اولى يكشف ايراني احجام ايراني متجهه اوروبا موقع زنغنه قوله شركه توتال فرنسيه ستشتري برميل ايران لمسات اخيره ستوضع جاري شركه ايني ايطاليه مهتمه بشرا برميل ايران مضيفا ممثلي شركه ايطاليه سيزورون طهران مستقبل قريب لمناقشه شروط زنغنه شركه ساراس ايطاليه لتكرير مهتمه بشرا برميل ايران تامل طهران استثمارات اجنبيه بنحو مليار دولار لاستثما صناعه ايرانيه استكشاف تنميه,أسواق_النفط,1,0
21356,سعوديه تمتلك احتياطيات عالم نفطيه موكده قالت مصادر مطلعه مراجعه مستقله لاحتياطيات آرامكو سعوديه اكدت بيانات شركه تشير احتياطيات تبلغ مليار برميل آرامكو طلبت شركتين امريكيتين متخصصتين مراجعه تقييم احتياطيات سعوديه اطار تهدف لطرح شركه مملوكه حكومه سعوديه اكتتاب مراجعه حقول ارامكو تحتوي احتياطيات عالميه موكده دلائل احتياطيات تفوق بكثير مستوى توثر قيمه سوقيه شركه ادراج مصدر مطلع مراجعه مستقله تكشف مفاجآت متوقع يصبح ادراج اكبر اولي عالم يمثل احدى ركائز حكومه سعوديه لتحقيق تحول مملكه استثمار تنويع موارد اقتصاد تقليص اعتماده ارامكو امير محمد سلمان يشرف سياسه طاقه سياسه اقتصاديه اكبر مصدر عالم رويه مملكه ماضي توقع محمد سلمان يقيم اولي ارامكو بتريليوني دولار اعرب اعتقاده سيرتفع نهايه مطاف فريد غايرلي,أسواق_النفط,1,0
21159,انتاج نفط كويتي يعود نصف معدله يومي عاد انتاج نفط كويت نصف معدله يومي بحدود مليون برميل مواصله عاملين قطاع اضرا افاد مسوولون كويتيون عمال قطاع نفط غاز اضرابا مفتوحا احتجاجا خطط حكوميه لجداول رواتب مخفضه لموظفي قطاع عام ادى خفض انتاج لنحو مليون برميل يوميا مقابل ثلاثه ملايين اعلن متحدث باسم قطاع نفطي شيخ طلال خالد صول انتاج متوسط قدر بنحو مليون برميل يوميا نفط خام بيان نشرته كاله انبا رسميه اشار خالد عوده انتاج نفط خام شركه نفط كويت معدلاته طبيعيه شمال كويت تشغيل مركز تجميع جنوب شرق بلاد زياده معدلات انتاج يحدد خالد سبل توفير عماله مطلوبه لانتاج نفط علما مجلس وزرا طلب موسسه بترول اتخاذ اجراات لازمه لضمان يوثر اضراب عمليات انتاج علما كويت تعد رابع منتج ضمن منظمه دول نفط اوبك اكد خالد قطاع تسويق عالمي يلبي احتياجات عملا ترد ايه تقارير جود تاخير جدوله لعمليات شحن نفط خام تصدير زير نفط وكاله انس صالح جدد مسا دعوته عمال عوده عمل متعهدا روات تخفض اتحاد عمال بترول صناعه بتروكيماويات نقابات عماليه يشددون اضراب مرهون بتخلي حكومه جداول رواتب جديده خطط خصخصه اجزا قطاع نفطي مسوولون قطاع نفطي مخزون دوله كويت بنزين مشتقات بتروليه يكفي لاستيفا حاجه بلاد لمده يوما مخزون استراتيجي دوله يكفي لمده يوما,أسواق_النفط,1,0
21331,قطع امدادات غاز طهران تهدد لجو محاكم دوليه لوحت شركه غاز وطنيه ايرانيه لجو تحكيم دولي لحل خلا تركمانستان متعلق ديون خاصه بامدادات غاز ايران تركمانستان تقطع غاز ايران تكشف سبب اعلنت شركه غاز وطنيه ايرانيه بيان سددت ديون جاريه قسما ديون سابقه لشركه تركمان غاز بنحو مليار دولار بحسن نيه عبر طرق مختلفه مشيره قامت بسداد ديون مترتبه فتره عقوبات دوليه مفروضه جرى اتفاق طرفين سداد طريق صادرات سلع خدمات ضمانات بقيمه مئات ملايين دولارات لمصدري سلع خدمات شركه غاز وطنيه ايرانيه اعتبر جانب ايراني مساعي لحل خلافات شركه تركمان غاز بات فشل يدفع شركه غاز وطنيه ايرانيه توجه لرفع قضيه محكمه تحكيم دوليه اصدرت شركه غاز وطنيه ايرانيه بيان بيان صادر زاره خارجيه تركمانستانيه ورد اسباب قطع امدادات غاز تركمانستان ايران يعود لعدم سداد اخيره ديون مستحقه ناديجدا انيوتينا,أسواق_النفط,1,0
6623,مدفيديف يدعو انطلاق واقع اعداد موازنه رئيس وزرا روسي دميتري مدفيديف حكومه مخاطر هبوط اسعار مستقبلا انطلاق توقعات اقعيه اعداد موازنه بلاد اعوام ثلاثه قادمه اجتماع حكومه روسيه بشان ميزانيه روسيه مدفيديف تحسن طفيف تشهده اسواق افضل اتباع مسار متحفظ تقييم توقعاتنا لاسعار مدفيديف اعضا حكومه روسيه تذبذبات ماضي حافظت اسعار مستويات متدنيه لترتفع دولارا برميل نهايه هبوط مجددا مدفيديف اسعار استقرت حاليا دولارا برميل مستوى يفوق توقعات ضعتها حكومه سابقا ارجعت كاله طاقه دوليه ارتفاع اسعار دولارا برميل تراجع امدادات نتيجه حرائق غابات كندا هجمات متمردين

In [7]:
#testing regex quirks in Arabic text
test1 = "الجزائر"
test2 = "الجزاىر"

test1.count(r"الجزا.ر")

#Wildcard not working

0

In [8]:
oil = r"برميل|نفظ|بترول"

@labeling_function()
def oil_terms(x):
    return POS if re.search(oil, x.text) else NEG

In [9]:
econ = r"سعر|اسعار|ستثمار|اقتصاد|سوق|اسواق"

@labeling_function()
def econ_terms(x):
    return POS if re.search(econ, x.text) else ABSTAIN

In [10]:
conflict = r"ضبرات|معارك|سلح"

@labeling_function()
def conflict_terms(x):
    return NEG if re.search(conflict, x.text) else ABSTAIN

In [11]:
exclude = r"كرة القدم|فلم|افلام"

@labeling_function()
def misc_exclude(x):
    return NEG if re.search(exclude, x.text) else ABSTAIN

In [12]:
countries = r"خليجي|الجزائر|ليبي|نيجيري|سعودي|روسي|امريك"

@labeling_function()
def oil_countries(x):
    return POS if re.search(countries, x.text) else ABSTAIN


In [13]:
lfs = [oil_terms, econ_terms, conflict_terms, misc_exclude, oil_countries]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=train)
L_dev = applier.apply(df=dev)

100%|██████████| 3542/3542 [00:01<00:00, 3422.43it/s]


In [14]:
from snorkel.labeling.analysis import LFAnalysis

LFAnalysis(L=L_dev, lfs=lfs).lf_summary(Y=Y_dev)

,j,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
oil_terms,0,"[0, 1]",1.000000,0.691982,0.585827,104,0,0.974026
econ_terms,1,[1],0.133258,0.133258,0.104178,143,329,0.302966
conflict_terms,2,[0],0.207228,0.207228,0.134387,725,9,0.987738
misc_exclude,3,[0],0.007340,0.007340,0.004517,26,0,1.000000
oil_countries,4,[1],0.576510,0.576510,0.549125,151,1891,0.073947


In [15]:
from snorkel.analysis.error_analysis import get_label_buckets

buckets = get_label_buckets(Y_dev, L_dev[:, 1])
dev.iloc[buckets[(POS, ABSTAIN)]].sample(5, random_state=1)

,text,category,oil,syria
21065,غارديان تتحصل ثائق داعش لبنا دوله خلافه نشرت صحيفه غارديان بريطانيه ثيقه سريه مسربه تعود لداعش تكشف تفاصيل خطط تنظيم ارهابي لبنا اسس يدعي دوله خلافه مرحله اولى اعلان تنظيم خلافه اسلاميه اراضي عراق سوريا ركز قادته اصدار اوامر تخص جوانب سطحيه حياه اجتماعيه حدود دوله مزعومه حظر ارتدا ملابس اسلاميه تربيه حمام اعتبار اخيره مضيعه وقت انتقل داعش اصدار لوائح تعليمات تخص دوله بحد نشرت غارديان ثيقه سريه مكونه صفحه تحمل عنوان مبادئ اداره دوله اسلاميه تضم وثيقه مذكوره مبادئ نظام تربوي اعلامي لداعش آليات دعايته علاقاته خارجيه تحكم تجاره نفط غاز تصور وثيقه نظاما شديد تعقيد لتسيير معسكرات تنقسم اعداديه خاصه اطفال تدريبيه لمسلحين محترفين تشير صحيفه برنامج يبرز مدى جديه تعاطي تنظيم ارهابي قضيه اسس دوله غارديان تحدي يواجهه ائتلاف غربي داعش ينحصر تنفيذ مهمه قتاليه عاديه تنظيم اعتباره مجرد جمله مسلحين تنقل صحيفه ستينلي ماكريستال جنرالات احتياط امريكيين قوله غرب بتعامله داعش عصابه مجانين عاديين يجازف وقوع خطر تقدير قواه صحيفه بريطانيه تشير ثائق داعش صادره مدار اشهر خمسه اخيره تركز اجراات امن تعبئه تجنيد امر يدل تنامي اعراض بارانويا صفوف تنظيم فرض حظرا شبكات الـ خاصه اعلن عاما هاربين ساحات قتال نقص جنود تنظيم نوفوستي,أسواق_النفط,1,0
21414,مصر ثالث كشف غاز لشركه اعلنت شركه نفط بريطانيه اكتشافا جديدا غاز طبيعي منطقه امتياز شمال دمياط بحريه بشرق دلتا نيل يوكد جود مكامن لغاز منطقه مصريه يعد ثالث نجاح لنشاطـ قطاع كشفي سلامات اتول وتم حفر بئر استكشافيه اطلق اسم قطاميه ضحله بعمق اجمالى مترا مياه امتار تقريبا تدل معطيات اوليه جود طبقه حامله غاز بسماكه مترا تجري حاليا دراسه خيارات متعلقه بربط كشف بنيه تحتيه قائمه بوب دودلي رئيس تنفيذى لمجموعه عالميه يعتبر بئر قطاميه كشف منطقه امتياز نقوم فعل بتطوير حقل اتول تقييم اكتشاف سلامات نجاحنا مستمر مجال يوكد ايماننا دلتا نيل حوض غاز طراز عالمي تقع بئر قطاميه ضحله شمال مدينه دمياط تبعد جنوب غرب حقل سلامات غرب تسهيلات حقل حابى بحريه يذكر شركه تنتج نحو اجمالي غاز مصري فريد غايرلي,أسواق_النفط,1,0
21057,موسكو تعلق توريدات روسي اوكرانيا اعلن يكسي ميللر رئيس شركه غازبروم روسيه تعليق امدادات طبيعي روسي اوكرانيا استلام دفعات نقديه جديده كييف توريدات روسيا تعتزم ايقاف ارداتها لاوكرانيا تنظر تعليق امدادات رئيس شركه غازبروم روسيه شركته تتلق دفعات جديه شركه طاقه اوكرانيه نفطوغاز توريدات جديده تعليق امدادات بحلول ساعه بتوقيت موسكو روسيا اوكرانيا مفوضيه اوروبيه نهايه ماضي بروتوكول مبدئي بشان توريدات روسي اوكرانيا يعرف حزمه شتويه اتفاق قيام كييف بدفع مسبقا روسي مورد تضمن اتفاق كييف تخفيضات توفير تمويل لازم لاوكرانيا مفوضيه اوروبيه لشرا كميات مطلوبه طبيعي لازم مستودعات اوكرانيه بهدف ضمان امدادات مستقره روسي اوروبا اراضي اوكرانيه,أسواق_النفط,1,0
21133,شركه بريطانيه تسحب موظف جزائر كشفت شركه بريتش بتروليوم ستسحب موظف محطتي عين صالح ميناس غاز جزائر اسبوعين مقبلين اعلنت شركه نفط غاز نرويجيه شتات اويل تشغل منشاتين شراكه خطط لتخفيض موظف هجوم قذائف صاروخيه منشاه عين صالح يسفر قوع اصابات خسائر قالت شركه بريطانيه بيان قررت اجرا نقل موقت مراحل لجميع افراد طاق مشروعي عين صالح اميناس جزائر اسبوعين مقبلين اتخذ قرار كاجرا احترازي وقت ماضي استهدف مسلحون مجهولون قاعده نفطيه بعين صالح جزائر صواريخ انبا قوع ضحايا نقلت صحيفه خبر جزائريه مصادر امنيه عناصر جيش وطني شعبي تصدت لمحاوله اعتدا ارهابي منشاه نفطيه اقعه منطقه خريشبه كلم عين صالح اضافت جماعه ارهابيه استهدفت قاعده نفطيه تابعه شركه بريطانيه بريتيش بتروليوم قذائف تمهيدا استيلا تدخل عناصر قوات جيش وطني شعبي احبط محاوله,أسواق_النفط,1,0
6757,تشغيل آبار طبيعي جديده متوسط كشفت زاره بترول مصريه نجاح ايني ايطاليه بريتش بتروليوم بتشغيل آبار جديده طبيعي متوسط اكدت وزاره مليون مكعب يوميا انتاج بنهايه ماضي بفضل تمكن شركات اجنبيه مذكوره تشغيل آبار اربعه متوسط دلتا صحرا غربيه افاد بيان وزاره آبار اربعه بمنطقه شمال دلتا تابع لشركه بمعدل انتاج مليون مكعب يوميا بمعدل انتاج مليون مكعب يوميا علما بئرين تابعين لحقل نورس تشرف ساهم بزياده انتاج ليصل مليون مكعب يوميا بمعدل انتاج مليون مكعب يوميا رابع سترا صحرا غربيه بمعدل انتاج مليون مكعب يوميا يذكر متوسط انتاج يبلغ حوالي مليار مكعب يوميا تستهلك بلاد حوالي مليار مكعب تعوض استيراد تراجع انتاج زياده استهلاك ايام دوله مصدره طاقه تحويل امدادات طاقه محليه مستورده طاقه باستيراد طبيعي مسال,أسواق_النفط,1,0


Hm...a few entries here are definitely not about the oil market. Bad labeled data! What an unusual problem!

Other errors seem to indicate we can just expand the keywords in existing LFs and get better results.

In [16]:
oil = r"غاز|برميل|نفظ|بترول"
econ = r"سعر|اسعار|ستثمار|اقتصاد|سوق|اسواق|شركة"
conflict = r"ضربات|اهلي|عرقي|معارك|سلح|ارهاب|متطرف"
countries = r"فنزو|كويت|اران|ابو ظبي|قطر|عراق|خليجي|الجزائر|ليبي|نيجيري|سعودي|روسي|امريك|برازيل|امارات"

In [17]:
#Let's add a few more functions as well
@labeling_function()
def discover_oil(x):
    return POS if re.search(oil, x.text) and re.search("كشف|اكشاف", x.text) else ABSTAIN

companies = "بي بي|شيفرون|كونوكو فيلبس|توتال|اكسون|موبيل|شركه نفظ|سوناتراك|روسنفت|وطني للغاز|وطني للبترول|شركه بترول"

@labeling_function()
def oil_companies(x):
    return POS if re.search(companies, x.text) else ABSTAIN

@labeling_function()
def economy_stupid(x):
    return POS if re.search(oil, x.text) and re.search(econ, x.text) else ABSTAIN 

In [18]:
lfs = [oil_terms, econ_terms, conflict_terms,
       misc_exclude, oil_countries, discover_oil,
      oil_companies, economy_stupid]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=train)
L_dev = applier.apply(df=dev)
L_valid = applier.apply(df=valid)
L_test = applier.apply(df=test)



100%|██████████| 5534/5534 [00:03<00:00, 1631.14it/s]


In [19]:
LFAnalysis(L=L_dev, lfs=lfs).lf_summary(Y=Y_dev)

,j,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
oil_terms,0,"[0, 1]",1.000000,0.786279,0.635517,129,0,0.954263
econ_terms,1,[1],0.133258,0.133258,0.097685,143,329,0.302966
conflict_terms,2,[0],0.341615,0.341615,0.246189,1197,13,0.989256
misc_exclude,3,[0],0.007340,0.007340,0.004517,26,0,1.000000
oil_countries,4,[1],0.647939,0.647939,0.601073,158,2137,0.068845
discover_oil,5,[1],0.006494,0.006494,0.001976,12,11,0.521739
oil_companies,6,[1],0.007905,0.007905,0.006776,6,22,0.214286
economy_stupid,7,[1],0.038679,0.038679,0.003106,105,32,0.766423


In [20]:
from snorkel.labeling.model import MajorityLabelVoter
from snorkel.labeling.model import LabelModel

majority_model = MajorityLabelVoter()
Y_pred_train = majority_model.predict(L=L_train)

label_model = LabelModel(cardinality=2, verbose=True)
label_model.fit(L_train=L_train, n_epochs=1000, lr=0.001, log_freq=100, seed=123)

majority_acc = majority_model.score(L=L_valid, Y=Y_valid)["accuracy"]
print(f"{'Majority Vote Accuracy:':<25} {majority_acc * 100:.1f}%")
label_model_acc = label_model.score(L=L_valid, Y=Y_valid)["accuracy"]
print(f"{'Label Model Accuracy:':<25} {label_model_acc * 100:.1f}%")

Majority Vote Accuracy:   74.8%
Label Model Accuracy:     87.6%


Note that we see the jump in performance that snorkel promises with the labeling model, probably because we have more LFs and none of them have perfect coverage/accuracy alone.

In [21]:
from snorkel.labeling.utils import filter_unlabeled_dataframe
from snorkel.analysis.utils import probs_to_preds

Y_probs_train = label_model.predict_proba(L=L_train)

train_filtered, Y_probs_train_filtered = filter_unlabeled_dataframe(
    X=train, y=Y_probs_train, L=L_train
)

Y_preds_train_filtered = probs_to_preds(probs=Y_probs_train_filtered)

In [22]:
#I did some quick testing here and Tfidf w bigrams worked best

from sklearn.feature_extraction.text import TfidfVectorizer

words_train = [row.text for i, row in train_filtered.iterrows()]
words_valid = [row.text for i, row in valid.iterrows()]
words_test = [row.text for i, row in test.iterrows()]

vectorizer = TfidfVectorizer(ngram_range=(2,2))
X_train = vectorizer.fit_transform(words_train)
X_valid = vectorizer.transform(words_valid)
X_test = vectorizer.transform(words_test)

In [23]:
from sklearn.naive_bayes import MultinomialNB

classifier = MultinomialNB().fit(X_train, Y_preds_train_filtered)
classifier.score(X=X_test, y=Y_test)

0.9645825804119985

Compare against accuracy when using the actual training data

In [24]:
classifier = MultinomialNB().fit(X_train, Y_train)
classifier.score(X=X_test, y=Y_test)

0.9624141669678352